# seq2seq
    many to many로, delay를 허용하는 버전, delay를 허용하지 않고 실시간으로 출력을 내놓는 버전으로 나뉨.

# 일반적인 RNN구조
    현재의(t시점의) input vector와 previous hidden state vector(t-1시점의)을 사용하여(가중 결합으로) t시점의 hidden state vector을 만들어냄.
	가중치는 모든 시점에서 동일함. 은닉층이 늘어나면 층별로는 달라짐.
	RNN에서는 활성화 함수를 tanh를 쓰는데 평균이 0으로 나오는 zero-centered activation function이라서 씀. RNN에서는 zero centered 가 중요.
	back propagation을 수행할 때는 t시점의 input vector까지 back prop이 될 뿐만 아니라 t-1시점, t-2시점 ... 과거의 input vector까지 모두 거슬러서 back prop이 진행된다.
	이렇게 되면 너무 연산량이 많아지므로 truncate 해서 특정 지점까지만 back prop을 하기도 함.

	RNN의 식은 ht = axt + bht-1 + c 인데 (t시점의 input, t-1시점의 hidden vector) 이 꼴은 계속 시점마다 이어진다. ht+1 = axt+1 + bht + c.
	ht로만 봤을 때 b의 거듭제곱을 해주는 꼴이 된다. 그렇다면 back prop에서 계속 b를 곱해줘야할텐데 b의 절대값이 1보다 작으면 gradient vanishing이, 1보다 크면 gradient explosion이 일어날 수 있다.
	기간이 늘어날수록 예측력 떨어짐(long term dependency)
	-> 해결을 위해 LSTM이 등장

# LSTM
    RNN이 오래된 기울기는 소실되므로 이것을 해결하기 위해 long short term memory가 등장 -> 덧셈이 추가되어 기울기 소실 x
	cell state를 추가하여 이전의 정보를 그대로 전달하는데 중간에 시그모이드를 element wise로 곱하고(특정 정보를 얼마나 흘려보낼지를 결정) -> forget gate layer
	새로운 cell state를 업데이트하기 위해 ht-1과 xt의 sigmoid와 tanh를 element wise로 곱하여 (새로운 정보를 얼마만큼 살릴지 결정하는 연산) 전의 cell vector에 더해줌(이 덧셈으로 인해 back prop에서 거듭제곱을 하지 않고 기울기의 소실 없이 연산 가능) 이 연산들은 전부 element wise 연산임. dot가 아님
	hidden state vector는 cell vector에 tanh를 씌우고 input 데이터에 sigmoid를 씌운 값과 곱하는데(element wise multiplication) cell vector에 어느 부분을 살릴지 정해진다. 이 hidden state vector는 다음 시점의 hidden state vector로 쓰인다.
	작동과정 예시: 따옴표로 시작되어 다음 문장을 생성할 때 원래 입력이 -1이라면 정보를 업데이트 할 필요가 없으므로 1을 곱하고 0을 더함. 따옴표가 닫히면 정보를 업데이트 해야하므로 0을 곱하여 기억을 지우고 1을 더해 업데이트 함.
	cell state와 hidden vector의 역할을 구분하기 위해서 예를 들면 따옴표 안에 mea 뒤에 n이라는 글자가 나오기 위해서 따옴표 정보는 잊고 mea 뒤에 무엇이 나올지만 집중해야함.
	따라서 cell state에 tanh를 건 값에 hidden 에 sigmoid 건 값을 곱해주는 단계에서 hidden 에 sigmoid를 건 값을 0으로 설정하여 즉 output gate를 0으로 설정하여 따옴표에 해당하는 dimension에는 아무 정보도 없게 됨. 그러면 따옴표 정보에 현혹될 필요 없이 mea 뒤에 n이 오는 것에만 집중가능.
	-> 지금 필요없는 정보는 가린채로 hidden state vector을 만들 수 있음
	다른 예로는 문장을 계속입력하다가 cell state를 증가하며 계속 count를 해서(첫번째 공백1에서 그대로 유지하기 위해 1을 곱하고 공백하나를 추가할때 업데이트 하기위해 1을 더해 cell state 가 2가 됨-> 이런식으로 공백을 count) 
	cell state가 특정값이 되면(공백을 3개 만들고 코드를 작성해야한다면 cell이 3이 됐을 때) hidden state vector에 노출을 시킴. 그러면 다시 cell은 0으로 reset됨(원래 cell * 0 + 0 (잊어야하므로 *0, 업데이트 정보0)).
	조건문을 위해 괄호가 열리면 cell이 업데이트되고 다시 괄호가 닫히면 hidden state vector에 노출이 되어 cell이 reset됨

# GRU
    cell 과 hidden vector을 구분하지 않음. 새로 들어온 input h~t에 대해서 zt만큼 살리면(LSTM의 input gate) 이전 시점의 ht-1 에 대해서는 1-zt만큼 살림(forget gate).
	LSTM보다 경량화된 버전임 (연산이 빠름) -> 성능이 크게 떨어지지 않음
	h~t를 만들기 위해 이전 정보들 ht-1에 가릴건 가리기 위해 rt를 곱해줌. LSTM의 output gate의 역할.